# Exercício 2



## Enunciado

Para a viga em balanço da figura abaixo tratada como um problema de Estado Plano de Tensões determine a matriz de rigidez e o vetor de forças para as seguintes discretizações:

a) Com 2 elementos triangulares lineares conforme mostrado nesta figura.
b) Com 1 elemento retangular com os nós 1,2,3 e 4.

Determine os deslocamentos livres considerando os nós 3 e 4 presos(deslocamentos nulos). 

Determine também os valores das tensões, segundo o sistema de eixos indicado nos pontos nodais e nos pontos centrais dos elementos considerados. Considere o carregamento indicado nesta figura com a distribuição de carga onde p = 150*(N+1)/20 . Adotar E (200.000) e Poisson(0.3) indicados na figura. Atente para trabalhar com unidades coerentes e que a espessura também está dada (t = 1). Utilize para as medidas do domínio o valor a = b = 4 + (N-1)/20


![](ex2.png)

## Resolução

## Implementação

## Resultados